# TF-IDF e busca por conteúdo

Nesta atividade, vamos lidar com a seguinte situação: temos um grande banco de dados com textos, e queremos encontrar qual texto é mais relevante para uma consulta. Esse problema aparece em buscadores como Google, e também em sistemas locais como ElasticSearch.

In [1]:
import pandas as pd

DATASET = 'datasets/wikipedia_movies.zip'
df = pd.read_csv(DATASET).sample(1000)
df = df[['Title', 'Plot']]
print(df.head(), len(df))

                               Title  \
12888           Zarkorr! The Invader   
21897          The Little Kidnappers   
25011  Dulhan Wahi Jo Piya Man Bhaye   
28684             Mumbai-Pune-Mumbai   
23642                      Rebellion   

                                                    Plot  
12888  Intelligent aliens who have been studying Eart...  
21897  Coralee Elliott Testar's version of the story ...  
25011  Seth Harikrishan, the industrialist had as his...  
28684  The story revolves around two characters, a bo...  
23642  Notorious of its unpeace, the South district (...   1000


## Exercício 1
**Objetivo: lembrar-se do que é TF e o que é DF**

Identifique o Term Frequency e o Document Frequency nas asserções abaixo:

1. Quanto maior o ___, mais comum é a palavra entre os documentos de uma coleção
1. Quanto maior o ___, mais vezes a palavra é mencionada num documento específico
1. $P(w | \text{documento})$
1. $P(w | \text{coleção})$
1. Ajuda a identificar a coleção da qual um documento faz parte
1. Ajuda a identificar um documento dentro de uma coleção

1: DF
2: DF
3: TF
4: DF
5: DF
6: TF

## Exercício 2
**Objetivo: refletir sobre o uso de TF-IDF**

A medida TFIDF diz o quão relevante um documento é dentro de uma coleção e em relação a uma palavra específica. Ela é calculada para um par palavra-documento como:

$\text{TFIDF = TF / DF}$

Quando um documento tem um TFIDF alto em relação a uma palavra, isso significa que:

1. A palavra tende a ser (comum / incomum)
1. O documento menciona a palavras (muitas / poucas) vezes

Portanto, qual seria uma maneira de escrever um documento que tem intencionalmente um TFIDF alto para uma palavra?

## Exercício 3
**Objetivo: calcular TFIDF para documentos usando sklearn**

TFIDF pode ser entendido como um processo de vetorização, semelhante a usar o CountVectorizer. Abaixo, há um código que mostra um exemplo dessa vetorização usando sklearn. 

1. Escolhendo um filme aleatório da coleção que carregamos, identifique o TFIDF das palavras "zombie", "fungus" e "survival".
1. Identifique o filme que tem o maior TFIDF para a palavra "zombie".

In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

vectorizer = TfidfVectorizer(max_features=10000)
tfidf = vectorizer.fit_transform(df['Plot'])
print(vectorizer.vocabulary_)


{'farmer': 3293, 'chris': 1690, 'lowe': 5316, 'meets': 5654, 'and': 405, 'falls': 3274, 'in': 4324, 'love': 5306, 'with': 9857, 'molly': 5831, 'chorus': 1685, 'girl': 3722, 'despite': 2481, 'the': 9013, 'fact': 3252, 'that': 9011, 'she': 8126, 'is': 4570, 'city': 1725, 'through': 9074, 'accepts': 97, 'his': 4116, 'proposal': 6890, 'of': 6189, 'marriage': 5542, 'after': 201, 'wedding': 9755, 'goes': 3751, 'to': 9128, 'live': 5217, 'on': 6222, 'farm': 3292, 'realises': 7179, 'transition': 9236, 'for': 3483, 'will': 9822, 'be': 869, 'difficult': 2560, 'an': 395, 'attempt': 689, 'ease': 2908, 'her': 4061, 'into': 4523, 'life': 5165, 'buys': 1353, 'calf': 1377, 'look': 5272, 'unfortunately': 9419, 'finds': 3389, 'rural': 7719, 'extremely': 3238, 'does': 2717, 'not': 6124, 'settle': 8057, 'down': 2771, 'fails': 3259, 'local': 5234, 'community': 1913, 'starts': 8502, 'feel': 3330, 'has': 3980, 'made': 5377, 'big': 998, 'mistake': 5796, 'tries': 9280, 'quell': 6992, 'by': 1355, 'spending': 842

In [3]:
n_filme = 0
j = vectorizer.vocabulary_['assistant']
print(df.iloc[n_filme]['Title'], tfidf[n_filme, j])

Strawberry Roan 0.0


In [4]:
TERMO_DE_BUSCA = "death"
j =  vectorizer.vocabulary_[TERMO_DE_BUSCA]
m = 0
m_max = 0
m_i = 0
for n_filme in range(tfidf.shape[0]):
    m = tfidf[n_filme, j]
    if m>m_max:
        m_max = m
        m_i = n_filme

print(df.iloc[m_i]['Title'], tfidf[m_i, j])
print(df.iloc[m_i]['Plot'])

The Last Mile 0.18903635530879428
The movie presents the life in a prison where men are on death row. Some of them are wrongfully accused and convicted, there is nothing else in their future but the electric chair.
Richard Walters is condemned to death for crime he claims he never committed. While the drama inside the prison unfolds, his friends on the outside are trying to find evidence that he is innocent.[2]


In [5]:
TERMO_DE_BUSCA = "death"
j =  vectorizer.vocabulary_[TERMO_DE_BUSCA]
local_tfidf = tfidf[:,j]
m_i = np.argmax(local_tfidf)
m_max = local_tfidf[m_i]

print(df.iloc[m_i]['Title'], tfidf[m_i, j])
print(df.iloc[m_i]['Plot'])

The Last Mile 0.18903635530879428
The movie presents the life in a prison where men are on death row. Some of them are wrongfully accused and convicted, there is nothing else in their future but the electric chair.
Richard Walters is condemned to death for crime he claims he never committed. While the drama inside the prison unfolds, his friends on the outside are trying to find evidence that he is innocent.[2]


## Exercício 4
**Objetivo: implementar uma busca por vários termos simultaneamente**

Uma possível maneira de implementar uma busca por vários termos é somar o TFIDF de todas as palavras da query para cada documento da coleção, e então retornar o documento que tem a maior soma. Por exemplo, numa busca por "zombie fungus survival" deveríamos somar, para cada documento, o TFIDF de "zombie", de "fungus" e de "survival" e então ordenar o resultado.

1. Escreva código que implemente uma busca na base de dados de filmes à partir de uma query específica.
1. Qual é a complexidade ($O(...)$) da sua busca?

In [6]:
import re
query = "zombie fungus survival"

palavras = re.findall("\w+", query)
# print(palavras)
# Implemente sua solução aqui
j = [vectorizer.vocabulary_[t] for t in palavras if t in vectorizer.vocabulary_.keys()]
local_tfidf = tfidf[:,j].todense()
local_tfidf = np.min(local_tfidf, axis=1)

m_i = np.argmax(local_tfidf)
m_max = local_tfidf[m_i]

print(df.iloc[m_i]['Title'], tfidf[m_i, j])
print("----")
print(df.iloc[m_i]['Plot'])

Dawn of the Dead   (0, 0)	0.1805249049557816
----
After finishing a long shift as a nurse, Ana returns to her suburban neighborhood and her husband, Luis. Caught up in a scheduled date night, they miss an emergency news bulletin. The next morning, a neighborhood girl enters their bedroom and kills Luis, who immediately reanimates as a zombie and attacks Ana. She flees in her car, crashes, and passes out. Upon waking, Ana joins police sergeant Kenneth Hall, electronics salesman Michael, petty criminal Andre and his pregnant wife, Luda. They break into a nearby mall and are attacked by a zombified security guard, who scratches Luda. Three living guards — C.J., Bart, and Terry — make them surrender their weapons in exchange for refuge. They split into groups to secure the mall. On the roof, they see another survivor, Andy, who is stranded in his gun store across the zombie-infested parking lot.
The next day, a delivery truck carrying more survivors enters the lot, pursued by zombies. C.J.

In [7]:
j =  vectorizer.vocabulary_[TERMO_DE_BUSCA]
local_tfidf = tfidf[:,j]

m_i = np.argmax(local_tfidf)
m_max = local_tfidf[m_i]

print(df.iloc[m_i]['Title'], tfidf[m_i, j])
print(df.iloc[m_i]['Plot'])

The Last Mile 0.18903635530879428
The movie presents the life in a prison where men are on death row. Some of them are wrongfully accused and convicted, there is nothing else in their future but the electric chair.
Richard Walters is condemned to death for crime he claims he never committed. While the drama inside the prison unfolds, his friends on the outside are trying to find evidence that he is innocent.[2]


## Exercício 5
**Objetivo: implementar um índice invertido**

Você provavelmente reparou (talvez não tenha reparado, e é tudo bem) que, para fazer a busca, até agora, teve que varrer todos os documentos da sua coleção. Isso provavelmente levaria algum tempo, especialmente quando a coleção começa a aumentar. Para evitar ter que varrer todos os documentos da coleção, podemos implementar uma técnica chamada *índice invertido*. A ideia do índice invertido é usar um dicionário cujas chaves são as palavras do vocabulário e cujo conteúdo é uma lista de documentos que contém essa palavra, possivelmente acompanhados do TFIDF correspondente. Por exemplo:

In [8]:
indice = { 'palavra_1' : {'documento_1': 0.5, 'documento_2': 0.1}, 
          'palavra_2' : {'documento_2': 0.6},
            'equalization' : {'documento_3': 0.7}    }

def buscar(palavras, indice):
    assert type(palavras)==list
    resultado = dict()
    for p in palavras:
        if p in indice.keys():
            for documento in indice[p].keys():
                if documento not in resultado.keys():
                    resultado[documento] = indice[p][documento]
                else:
                    resultado[documento] += indice[p][documento]
    return resultado

buscar(['palavra_1', 'palavra_2', 'equalization'], indice)#retornando a relevancia de documentos para a query

{'documento_1': 0.5, 'documento_2': 0.7, 'documento_3': 0.7}

1. Adicione uma nova palavra ao índice e escolha seu TFIDF. Realize uma nova busca e verifique o resultado.
1. Escreva uma função que ordena o resultado e retorna apenas `N` documentos mais relevantes para sua busca.
1. Incremente sua biblioteca de forma que ela passe a receber uma string como entrada (representando a query) e retorne os `N` documentos mais relevantes (`N` pode ser definido arbitrariamente).

In [9]:
# versao original alterada para ter funcao de encontrar n_maiores
indice = { 'palavra_1' : {'documento_1': 0.5, 'documento_2': 0.1}, 
          'palavra_2' : {'documento_2': 0.6}    }

def buscar(palavras, indice):
    assert type(palavras)==list
    resultado = dict()
    for p in palavras:
        if p in indice.keys():
            for documento in indice[p].keys():
                if documento not in resultado.keys():
                    resultado[documento] = indice[p][documento]
                else:
                    resultado[documento] += indice[p][documento]
    return resultado

res = buscar(['palavra_1', 'palavra_2'], indice)#retornando a relevancia de documentos para a query

def n_maiores(res_busca, n):
    res = []
    for k in res_busca.keys():
        res.append((res_busca[k],k))
    res = sorted(res, reverse = True)[0:n]
    return res

print(n_maiores(res, 2))

def query(query_string, n, indice):
    palavras = re.findall('\w+', query_string)
    res = buscar(palavras, indice)
    res_n = n_maiores(res, n)
    return res_n

print(query("palavra_1 palavra_2 equalization", 2, indice))

[(0.7, 'documento_2'), (0.5, 'documento_1')]
[(0.7, 'documento_2'), (0.5, 'documento_1')]


## Exercício 6
**Objetivo: implementar um buscador de filmes**

Implemente uma função que recebe como entrada uma query e retorna os títulos e enredos dos 5 filmes mais relevantes para aquela query. Se precisar, use mais parâmetros ou variáveis globais. Teste a sua função e veja se você concorda com os resultados, incluindo se você consegue encontrar seus filmes favoritos e se consegue alguma recomendação relevante a um filme novo.

In [10]:
from tqdm import tqdm
indice_filmes = dict()

for w in tqdm(vectorizer.vocabulary_.keys()):
    indice_filmes[w] = dict()
    for j in range(tfidf.shape[0]):
        if(tfidf[j, vectorizer.vocabulary_[w]])>0:
            indice_filmes[w][j] = tfidf[j, vectorizer.vocabulary_[w]]
def query_movies(query : str):
    pass

100%|██████████| 10000/10000 [01:14<00:00, 134.22it/s]


In [11]:
res = query("king", 3, indice_filmes)
for n in res:
    print(df.iloc[n[1]]['Title'])
    print(df.iloc[n[1]]['Plot'])
    print("--")

Amber
Amber (Baby Tanuja), a young orphaned tribal girl, stays with her maternal grandfather who is the Chief. She learns that her father was a prince who had married her mother but was killed. The cause of the murder was unknown and the killers were never caught. Her mother had committed suicide soon after. The grandfather sends her to the palace to stay with her paternal grandfather, the King (Bipin Gupta). The King comes to love Amber and she grows up (Nargis) surrounded by love and luxury. However, she is let known through palace intrigue that her grandfather, the King, had got her father killed. She decides to avenge her father's death by killing her grandfather. Ambar, on one of her outings meets Raj (Raj Kapoor), and the two fall in love. Raj turns out to be a bandit, but his father is a loyal server to the king. Raj's father fears that someone is going to harm the king so he sends Raj to the palace. Raj arrives there pretending to be a teacher.
The King's minister, Diwanji (Ram

## Exercício 7
**Objetivo: identificar palavras-chave usando TFIDF**

Uma outra aplicação de TFIDF é encontrar palavras-chave, isto é, palavras que diferenciam um documento do restante dos documentos de sua coleção.

Incremente seu buscador de forma que, além do título e enredo, ele também escolha as algumas palavras (escolha quantas!) mais relevantes de cada documento e as imprima como keywords.

## Exercício 8
**Objetivo: encontrar documentos semelhantes usando TFIDF**

Uma maneira de encontrar documentos semelhantes em uma coleção de textos é assumir que o texto do documento é uma query, e então realizar a busca normalmente. O problema disso é que provavelmente teríamos textos muito longos e a query ficaria muito carregada. Para solucionar isso, poderíamos usar apenas as palavras mais relevantes de um documento como query. Implemente uma função que recebe o índice (ou outro identificador único) de um documento de nosso banco de dados e então encontra 5 documentos semelhantes a ele.